In [3]:
!pip install geopy

    100% |████████████████████████████████| 112kB 4.4MB/s ta 0:00:01


In [4]:
import configparser
from datetime import datetime
import os
from datetime import timedelta
import pandas as pd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from functools import partial

In [5]:
data = pd.read_csv('airports.csv')

In [6]:
data.dtypes

id                     int64
ident                 object
type                  object
name                  object
latitude_deg         float64
longitude_deg        float64
elevation_ft         float64
continent             object
iso_country           object
iso_region            object
municipality          object
scheduled_service     object
gps_code              object
iata_code             object
local_code            object
home_link             object
wikipedia_link        object
keywords              object
dtype: object

In [8]:
# selecting the airport code for US only

data = data.where(data.iso_country =='US').dropna(axis=0,how='all')
data.head()

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords
0,6523.0,00A,heliport,Total Rf Heliport,40.070801,-74.933601,11.0,NaN,US,US-PA,Bensalem,no,00A,NaN,00A,NaN,NaN,NaN
1,323361.0,00AA,small_airport,Aero B Ranch Airport,38.704022,-101.473911,3435.0,NaN,US,US-KS,Leoti,no,00AA,NaN,00AA,NaN,NaN,NaN
2,6524.0,00AK,small_airport,Lowell Field,59.949200,-151.695999,450.0,NaN,US,US-AK,Anchor Point,no,00AK,NaN,00AK,NaN,NaN,NaN
3,6525.0,00AL,small_airport,Epps Airpark,34.864799,-86.770302,820.0,NaN,US,US-AL,Harvest,no,00AL,NaN,00AL,NaN,NaN,NaN
4,6526.0,00AR,closed,Newport Hospital & Clinic Heliport,35.608700,-91.254898,237.0,NaN,US,US-AR,Newport,no,NaN,NaN,NaN,NaN,NaN,00AR


In [9]:
# function to split the iso-region values to country and statecode

lambda x: x.split('-')[1]

<function __main__.<lambda>(x)>

In [10]:
# new column statecode

data['statecode'] = data['iso_region'].apply(lambda x: x.split('-')[1])

In [11]:
data.head()

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords,statecode
0,6523.0,00A,heliport,Total Rf Heliport,40.070801,-74.933601,11.0,NaN,US,US-PA,Bensalem,no,00A,NaN,00A,NaN,NaN,NaN,PA
1,323361.0,00AA,small_airport,Aero B Ranch Airport,38.704022,-101.473911,3435.0,NaN,US,US-KS,Leoti,no,00AA,NaN,00AA,NaN,NaN,NaN,KS
2,6524.0,00AK,small_airport,Lowell Field,59.949200,-151.695999,450.0,NaN,US,US-AK,Anchor Point,no,00AK,NaN,00AK,NaN,NaN,NaN,AK
3,6525.0,00AL,small_airport,Epps Airpark,34.864799,-86.770302,820.0,NaN,US,US-AL,Harvest,no,00AL,NaN,00AL,NaN,NaN,NaN,AL
4,6526.0,00AR,closed,Newport Hospital & Clinic Heliport,35.608700,-91.254898,237.0,NaN,US,US-AR,Newport,no,NaN,NaN,NaN,NaN,NaN,00AR,AR


In [12]:
# selecting the required columns

data =data[['ident','type','name','elevation_ft','iso_country','iso_region','municipality',
             'gps_code','iata_code','local_code','statecode','latitude_deg','longitude_deg']]

In [14]:
data.head()

,ident,type,name,elevation_ft,iso_country,iso_region,municipality,gps_code,iata_code,local_code,statecode,latitude_deg,longitude_deg
0,00A,heliport,Total Rf Heliport,11.0,US,US-PA,Bensalem,00A,NaN,00A,PA,40.070801,-74.933601
1,00AA,small_airport,Aero B Ranch Airport,3435.0,US,US-KS,Leoti,00AA,NaN,00AA,KS,38.704022,-101.473911
2,00AK,small_airport,Lowell Field,450.0,US,US-AK,Anchor Point,00AK,NaN,00AK,AK,59.949200,-151.695999
3,00AL,small_airport,Epps Airpark,820.0,US,US-AL,Harvest,00AL,NaN,00AL,AL,34.864799,-86.770302
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,US,US-AR,Newport,NaN,NaN,NaN,AR,35.608700,-91.254898


In [15]:
# creating a new column called coordinate from the latitude and longitude column

data['coordinate'] = data['latitude_deg'].map(str) + ',' + data['longitude_deg'].map(str)
data.head()

,ident,type,name,elevation_ft,iso_country,iso_region,municipality,gps_code,iata_code,local_code,statecode,latitude_deg,longitude_deg,coordinate
0,00A,heliport,Total Rf Heliport,11.0,US,US-PA,Bensalem,00A,NaN,00A,PA,40.070801,-74.933601,"40.07080078125,-74.93360137939453"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,US,US-KS,Leoti,00AA,NaN,00AA,KS,38.704022,-101.473911,"38.704021999999995,-101.473911"
2,00AK,small_airport,Lowell Field,450.0,US,US-AK,Anchor Point,00AK,NaN,00AK,AK,59.949200,-151.695999,"59.94919968,-151.695999146"
3,00AL,small_airport,Epps Airpark,820.0,US,US-AL,Harvest,00AL,NaN,00AL,AL,34.864799,-86.770302,"34.86479949951172,-86.77030181884766"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,US,US-AR,Newport,NaN,NaN,NaN,AR,35.608700,-91.254898,"35.6087,-91.254898"


In [16]:
# method for geographical decoding. to get the city, county, street, state, e.t.c from the coordinate

def geocode(data):
    addresses =[]
    locator = Nominatim(user_agent="irocharles22@gmail.com",timeout=10)
    location = RateLimiter(locator.reverse, min_delay_seconds=0.001)
    for i in data['coordinate']:
        reverse = location(i).raw
        addresses.append(reverse['address'])
    
    return addresses
    
    

In [17]:
# geocoding with the OpenStreetMap Nominatim geocoding services.

data['geocode'] = geocode(data)

In [18]:
# saving new csv file with city attribute to enable drilling down to city.

data.to_csv('Airportgeocode.csv')